In [2]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import geopandas as gpd
import math
import json
from geopy.geocoders import Nominatim
from pyproj import Transformer
import os


In [22]:
geolocator = Nominatim(user_agent="bnbviz")
location = geolocator.geocode('5 rue de charonne, 75011 Paris, france')
radius = 3
x, y = location.latitude, location.longitude
xmin = x - radius / (2 * 110.574)
xmax = x + radius / (2 * 110.574)
ymin = y - radius / (2 * 111.320 * math.cos(math.pi * x / 180))
ymax = y + radius / (2 * 111.320 * math.cos(math.pi * x / 180))
transformer = Transformer.from_crs("epsg:4326", "epsg:2154")
xmin, ymin = transformer.transform(xmin, ymin)
xmax, ymax = transformer.transform(xmax, ymax)
xmin, ymin, xmax, ymax

(652553.2142296537, 6860055.072311686, 655582.3990020527, 6863047.99080445)

In [23]:
gdf = gpd.read_file('data/bnb_export.gpkg', bbox=(xmin, ymin, xmax, ymax))
gdf = gdf[
        [
            "geometry",
            "cerffo2020_annee_construction",
            "adedpe202006_mean_class_conso_ener",
            "adedpe202006_mean_conso_ener",
            "adedpe202006_mean_class_estim_ges",
            "adedpe202006_mean_estim_ges",
        ]
    ]
gdf.rename(
    columns={
        "cerffo2020_annee_construction": "Année de construction",
        "adedpe202006_mean_class_conso_ener": "Etiquette énergétique (DPE)",
        "adedpe202006_mean_conso_ener": "Consommations énergétiques, kWhEP/m².an (DPE)",
        "adedpe202006_mean_class_estim_ges": "Etiquette carbone (DPE)",
        "adedpe202006_mean_estim_ges": "Emissions de GES, kgC02eq/m².an (DPE)",
    },
    inplace=True,
)

In [10]:
import json

In [24]:
with open('../bdnbviz/default_gdf3.json', 'w') as file:
    data = json.loads(gdf.to_json())
    json.dump(data, file)